In [1]:
import numpy as np
import random

In [2]:
# Определение функции Химмельблау
def himmelblau(x, y):
    return (x**2 + y - 11)**2 + (x + y**2 - 7)**2

In [3]:
class PSO:
    def __init__(self, fitness_func, l_bound, h_bound, n_particles=20, n_dimensions=2, max_iter=100):
        self.l_bound = l_bound
        self.h_bound = h_bound
        self.fitness_func = fitness_func
        self.n_particles = n_particles
        self.n_dimensions = n_dimensions 
        self.max_iter = max_iter 
        # инициализируем списки для хранения данных о частицах и их лучших значениях
        self.particles = np.zeros((self.n_particles, self.n_dimensions))
        self.velocities = np.random.uniform(-1, 1, size=(self.n_particles, self.n_dimensions))
        self.pbest_positions = np.zeros((self.n_particles, self.n_dimensions))
        self.pbest_values = np.zeros(self.n_particles)
        # инициализируем глобальные лучшие значения
        self.gbest_position = np.zeros(self.n_dimensions)
        self.gbest_value = np.inf

    def initialize_particles(self,l_bound, h_bound):
        # инициализируем начальные значения для каждой частицы
        self.particles = np.random.uniform(l_bound, h_bound, size=(self.n_particles, self.n_dimensions))
        for i in range(self.n_particles):
            # сохраняем начальные значения как лучшие для каждой частицы
            self.pbest_positions[i] = self.particles[i]
            value = self.evaluate_fitness(self.fitness_func, self.particles[i])
            self.pbest_values[i] = value
            # проверяем, является ли начальное значение глобально лучшим
            if value < self.gbest_value:
                self.gbest_position = self.particles[i]
                self.gbest_value = value

    def update_particles(self):
       # Для подсчета скорости будем использовать 'глобальный подход'
       # v_i(t+1) = w*v_i(t) + c1*r1*(p_i - x_i) + c2*r2*(g - x_i)
       #  где v_i(t) - текущая скорость частицы i
       #  p_i - лучшее положение, которое достигала частица i
       #  x_i - текущее положение частицы i
       #  g - лучшее положение, которое достигали все частицы в рое
       #  w, c1, c2 - коэффициенты инерции, локального и глобального воздействия
       #  r1, r2 - случайные числа в диапазоне [0, 1]
       # обновляем значения скоростей и позиций для каждой частицы
        w=0.5       
        c1=1        
        c2=2        
        for i in range(self.n_particles):
            r1 = np.random.rand(self.n_dimensions)
            r2 = np.random.rand(self.n_dimensions)
            self.velocities[i] = w*self.velocities[i] + \
                c1 * r1 * (self.pbest_positions[i] - self.particles[i]) + \
                c2 * r2 * (self.gbest_position - self.particles[i])   
            self.particles[i] = self.particles[i] + self.velocities[i]
            # вычисляем значение функции для новой позиции и обновляем лучшее значение для частицы
            value = self.evaluate_fitness(self.fitness_func, self.particles[i])
            if value < self.pbest_values[i]:
                self.pbest_positions[i] = self.particles[i]
                self.pbest_values[i] = value
                # проверяем, не является ли новое значение глобально лучшим
                if value < self.gbest_value:
                    self.gbest_position = self.particles[i]
                    self.gbest_value = value

    def evaluate_fitness(self,fitness_func, position):
        x, y = position
        return fitness_func(x, y)

    def run(self):
        self.initialize_particles(self.l_bound, self.h_bound)
        for i in range(self.max_iter):
            self.update_particles()
        return self.gbest_position, self.gbest_value

In [6]:
pso = PSO(himmelblau,-5,5)
solution, val = pso.run()
print(f'X = {solution[0]:.4f}, Y = {solution[1]:.4f}, Z = {val}')

X = -2.8051, Y = 3.1313, Z = 2.270274850468683e-16
